#### Vanessa Navarro Coronado e Iván Sánchez Castellanos

# Práctica 3: Multiclasificadores y selección de variables

In [1]:
# Always load all scipy stack packages
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(color_codes=True)

In [2]:
# This code configures matplotlib for proper rendering
%matplotlib inline
mpl.rcParams["figure.figsize"] = "8, 4"
import warnings
warnings.simplefilter("ignore")

In [3]:
seed=6342
np.random.seed(seed)

## Carga de datos

Cargamos el dataset Wisconsin de la librería scikit.

In [4]:
from sklearn.datasets import load_breast_cancer
wisconsin = load_breast_cancer()

Estos datasets vienen preparados directamente para la clasificación y separan en un diccionario los distintos elementos que nos puedan interesar. Si quisieramos recuperar los datos en un data.frame tendríamos que combinar los elementos correctamente:

In [5]:
wisconsin.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [6]:
dfWisc = pd.DataFrame(
    data = np.column_stack((wisconsin.data, wisconsin.target)), 
    columns = np.append(wisconsin.feature_names, "label")
)

In [7]:
target_categorical = pd.Series(
     pd.Categorical([wisconsin.target_names[x] for x in wisconsin.target], 
     categories=wisconsin.target_names)
)

dfLabel = pd.DataFrame(target_categorical, columns=["label"])
dfLabel.sample(3)

,label
495,benign
67,benign
173,benign


Independientemente, realizaremos el resto de la práctica utilizando las versiones numéricas de los datos para demostrar su versatilidad.

In [8]:
attributesWisc = wisconsin.data
labelWisc = wisconsin.target

# 1. Uso y ajuste de métodos basados en ensembles

## 1.1 Implementación básica de un ensemble de manera manual:

## 1.2. Árbol de decisión como referencia

In [9]:
# Cargamos el arbol de decision
from sklearn import tree
from sklearn.model_selection import cross_val_score

dt = tree.DecisionTreeClassifier()
scores_dt = cross_val_score(estimator=dt, X=attributesWisc, y=labelWisc, scoring="accuracy", cv=3)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_dt.mean(), scores_dt.std() * 2))

Accuracy: 0.91 (+/- 0.05)


## 1.3. Bagging

In [10]:
from sklearn.ensemble import BaggingClassifier

bagg = BaggingClassifier(tree.DecisionTreeClassifier(), 
                          n_estimators = 30, 
                          random_state=seed)

In [11]:
scores_bagg = cross_val_score(bagg, attributesWisc, labelWisc, cv=3, scoring="accuracy")
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_bagg.mean(), scores_bagg.std() * 2))

Accuracy: 0.93 (+/- 0.04)


## 1.4. Random Forest

In [12]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=30, random_state=seed)

In [13]:
scores_rf = cross_val_score(rf, attributesWisc, labelWisc, cv=3, scoring="accuracy")
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_rf.mean(), scores_rf.std() * 2))

Accuracy: 0.95 (+/- 0.03)


## 1.5. Boosting

In [14]:
from sklearn.ensemble import AdaBoostClassifier
#boost = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(), n_estimators=30)
boost = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=3), n_estimators=30)

In [15]:
scores_boost = cross_val_score(boost, attributesWisc, labelWisc, cv=3, scoring="accuracy")
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_boost.mean(), scores_boost.std() * 2))

Accuracy: 0.97 (+/- 0.03)


## 1.6. Gradient Boosting

In [16]:
from sklearn.ensemble import GradientBoostingClassifier
gboost = GradientBoostingClassifier(n_estimators=30)

In [17]:
scores_gboost = cross_val_score(gboost, attributesWisc, labelWisc, cv=5, scoring="accuracy")
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_gboost.mean(), scores_gboost.std() * 2))

Accuracy: 0.95 (+/- 0.03)


# 2. Selección de variables

## 2.1. Utilizar un método filter basado en rankings y la importancia de las variables para evaluar distintos subconjuntos

## 2.2. Implementar al menos un algoritmo de búsqueda recursiva wrapper

## 2.3. Implementar un algoritmo greedy basado en métricas filter (información mútua)